# Data preparation

## Set up

In [1]:
import json
import pandas as pd
import numpy as np

## Load data

In [27]:
with open("MLA_100k.jsonlines", 'r') as file:
    data = [json.loads(line) for line in file]
df = pd.json_normalize(data)

# Goal: Extract main dictionaries from jsonlines
columns_to_keep_nested = ['seller_address', 'location', 'seller_contact', 'geolocation']
processed_data = []

for record in data:
    processed_record = {}
    for key, value in record.items():

        if key in columns_to_keep_nested:
            processed_record[key] = value 
        else:
            processed_record[key] = value
    processed_data.append(processed_record)

df = pd.DataFrame(processed_data)


In [28]:
df.shape

(100000, 48)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 48 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   seller_address                    100000 non-null  object 
 1   warranty                          39104 non-null   object 
 2   sub_status                        100000 non-null  object 
 3   condition                         100000 non-null  object 
 4   seller_contact                    2219 non-null    object 
 5   deal_ids                          100000 non-null  object 
 6   base_price                        100000 non-null  float64
 7   shipping                          100000 non-null  object 
 8   non_mercado_pago_payment_methods  100000 non-null  object 
 9   seller_id                         100000 non-null  int64  
 10  variations                        100000 non-null  object 
 11  location                          100000 non-null  ob

In [30]:
df.head(3)

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,geolocation,sold_quantity,available_quantity
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,active,None,NaN,None,1,2015-09-05T20:42:53.000Z,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,active,None,NaN,None,1,2015-09-26T18:08:30.000Z,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,active,None,NaN,None,1,2015-09-09T23:57:07.000Z,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1


### Target evaluation

In [31]:
condition_counts = df['condition'].value_counts()
condition_percentage = df['condition'].value_counts(normalize=True) * 100

condition_df = pd.DataFrame({
    'condition': condition_counts.index,
    'counts': condition_counts.values,
    'percentage': condition_percentage.values
})

condition_df

,condition,counts,percentage
0,new,53758,53.758
1,used,46242,46.242


## Data evaluation

### Non-features
- **seller_address**: [non-feature] It is a very specific data point with little chance of influencing the prediction.
- **listing_source**: [non-feature] Empty column.
- **site_id**: [non-feature] Contains the same data for all.
- **base_price**: [non-feature] Same base price data.
- **coverage**: [non-feature] Empty column.
- **international_delivery_mode**: [non-feature] Same data for all.
- **differential_pricing**: [non-feature] Empty column.
- **currency_id**: [non-feature] Non-relevant.
- **thumbnail**: [non-feature] Strongly related to pictures.
- **secure_thumbnail**: [non-feature] Strongly related to pictures.
- **catalog_product_id**: [non-feature] Only one record with data.
- **subtitle**: [non-feature] Empty column.
- **permalink**: [non-feature] Non-relevant.
- **geolocation**: [non-feature] Very specific.


### Maybe features
- **warranty**: [maybe] Text data will be ignored for now. However, in a future iteration, it can be extracted to provide information about whether the item is new or used.
- **category_id**: [maybe] String, to be encoded with one-hot encoding. There are over 10,000 categories, leading to many columns.
- **last_update**: [maybe]
- **date_created**: [maybe]
- **stop_time**: [maybe]
- **start_time**: [maybe]
- **title**: [maybe]
- **status**: [maybe]
- **non_mercado_pago_payment_methods**: [maybe]

### Candidate Features
- **sub_status**: [candidate] Categorical, to be one-hot encoded.
- **seller_contact**: [candidate] -> 'have_seller_contact': Create a boolean column.
- **deal_ids**: [candidate] -> 'have_deals_id': Create a boolean column.
- **base_price**: [candidate]
- **shipping**: [candidate] Extract the columns -> 'local_pick_up' and -> 'free_shipping'.
- **variations**: [candidate] -> 'have_variations': Create a boolean column.
- **locations**: [candidate] -> 'have_location': Create a boolean column.
- **listing_type_id**: [candidate] Categorical, to be one-hot encoded.
- **attributes**: [candidate] -> 'have_attributes': Create a boolean column.
- **buying_mode**: [candidate] Categorical, to be one-hot encoded.
- **tags**: [candidate] -> 'dragged_bids_and_visits', 'good_quality_thumbnail', 'dragged_visits', 'free_relist', 'poor_quality_thumbnail': Create a boolean column, 1 if data exists, 0 if not.
- **parent_item_id**: [candidate] -> 'have_parent_item': Create a boolean column.
- **descriptions**: [candidate] -> 'have_descriptions': Create a boolean column.
- **pictures**: [candidate] -> 'have_pictures': Create a boolean column.
- **official_store_id**: [candidate] -> 'have_official_store': Create a boolean column.
- **accepts_mercadopago**: [candidate] Convert True/False to 1/0.
- **original_price**: [candidate] -> 'have_original_price': Create a boolean column.
- **automatic_relist**: [candidate] -> 'have_automatic_relist': Create a boolean column.
- **video_id**: [candidate] -> 'have_video': Create a boolean column.
- **initial_quantity**: [candidate]
- **sold_quantity**: [candidate]
- **available_quantity**: [candidate]



## Data transformation

### Functions

In [50]:
def process_columns_with_empty_lists(dtf, columns):
    """
    Processes the specified columns in the DataFrame by converting them to strings, 
    replacing empty lists ('[]') with NaN, and creating a new boolean column indicating 
    whether the original column contains data or is missing.

    Parameters:
    dtf (pd.DataFrame): The DataFrame to be processed.
    columns (list): List of column names to be processed.

    Returns:
    pd.DataFrame: The modified DataFrame with additional boolean columns 
                  in the format 'have_{column_name}' for each processed column.

    Behavior:
    - Converts each specified column to string format.
    - Replaces any occurrence of '[]' (empty list representation) with NaN.
    - Creates a new column 'have_{column_name}' with a boolean indicator: 
      1 if the original column has data, 0 otherwise.
    - If a column is not found in the DataFrame, it prints a warning.
    
    """
    for col in columns:
        if col in dtf.columns:
            # Convert the column to string and replace empty lists with NaN
            dtf[col] = dtf[col].astype(str)
            dtf[col] = dtf[col].replace('[]', np.nan)
            # Create a boolean column indicating if there is data
            dtf[f'have_{col}'] = dtf[col].notnull().astype(int)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
    
    return dtf

In [51]:
def create_boolean_columns(dtf, columns):
    """
    Automates the creation of boolean columns indicating whether the specified columns
    contain data (not null). For each column, a new boolean column named 'have_{column_name}'
    is created, where 1 indicates the presence of data and 0 indicates missing data.

    Parameters:
    dtf (pd.DataFrame): The DataFrame to be processed.
    columns (list): List of column names to process.

    Returns:
    pd.DataFrame: The modified DataFrame with the new boolean columns.

    Example:
    columns_to_process = ['official_store_id', 'parent_item_id', 'video_id', 'original_price', 'seller_contact']
    dtf = create_boolean_columns(dtf, columns_to_process)
    """
    for col in columns:
        if col in dtf.columns:
            # Create a boolean column indicating if the original column has data
            dtf[f'have_{col}'] = dtf[col].notnull().astype(int)
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
    
    return dtf

### Process

In [55]:
dtf = df.copy()

In [56]:
columns_with_empty_list = ['deal_ids', 'variations', 'attributes', 'descriptions', 'pictures']
dtf = process_columns_with_empty_lists(dtf, columns_with_empty_list)

columns_to_boolean = ['official_store_id', 'parent_item_id', 'video_id', 'original_price', 'seller_contact']
dtf = create_boolean_columns(dtf, columns_to_boolean)

## One to one set up specific columns 
# accepts_mercadopago
dtf['accepts_mercadopago'] = dtf['accepts_mercadopago'].astype(int)

# automatic_relist
dtf['have_automatic_relist'] = dtf['automatic_relist'].astype(int)

# category_id 
dtf['category_id'] = dtf['category_id'].str.strip()

# shipping
dtf['local_pick_up'] = dtf['shipping'].apply(lambda x: x.get('local_pick_up'))
dtf['local_pick_up'] = dtf['local_pick_up'].astype(int)
dtf['free_shipping'] = dtf['shipping'].apply(lambda x: x.get('free_shipping'))
dtf['free_shipping'] = dtf['free_shipping'].astype(int)

# location
dtf['location'] = dtf['location'].astype(str)
dtf['location'] = dtf['location'].replace('{}', np.nan)
dtf['have_location'] = dtf['location'].notnull().astype(int)

# sub_status
dtf['sub_status'] = dtf['sub_status'].astype(str)
dtf['sub_status'] = dtf['sub_status'].str.strip()
dtf['sub_status'] = dtf['sub_status'].replace('[]', 'no_status')
dtf['sub_status'] = dtf['sub_status'].str.replace(r"[\[\]']", '', regex=True)

# tags
dtf['tags'] = dtf['tags'].apply(lambda x: x if isinstance(x, list) else [])
tags_dummies = dtf['tags'].apply(lambda x: pd.Series(1, index=x)).fillna(0).astype(int)
dtf = pd.concat([dtf, tags_dummies], axis=1)


In [60]:
dtf.buying_mode.value_counts()

buy_it_now    97001
classified     2219
auction         780
Name: buying_mode, dtype: int64

## Data to model

In [59]:
columns_to_model = ['id', 'condition','category_id','sub_status', 'have_seller_contact', 'have_deal_ids' ,'free_shipping', 'local_pick_up', 
                    'have_variations', 'have_location', 'listing_type_id', 'have_attributes', 'buying_mode', 'dragged_bids_and_visits', 
                    'good_quality_thumbnail', 'dragged_visits', 'free_relist', 'poor_quality_thumbnail', 'have_parent_item_id', 'have_descriptions', 
                    'have_pictures', 'have_official_store_id', 'accepts_mercadopago', 'have_original_price', 'have_automatic_relist', 'have_video_id',
                    'initial_quantity', 'sold_quantity', 'available_quantity', 'base_price']

df_to_model = dtf[columns_to_model]

categorical_columns = ['sub_status', 'listing_type_id', 'buying_mode', 'category_id']
df_onehot = pd.get_dummies(df_to_model[categorical_columns])

df_to_model_encoded = pd.concat([df_to_model, df_onehot], axis=1)
df_to_model_encoded.drop(categorical_columns, axis=1, inplace=True)

df_to_model_encoded['target_condition'] = df_to_model_encoded['condition'].apply(lambda x: 1 if x == 'new' else 0)
df_to_model_encoded 

,id,condition,have_seller_contact,have_deal_ids,free_shipping,local_pick_up,have_variations,have_location,have_attributes,dragged_bids_and_visits,...,category_id_MLA9984,category_id_MLA9988,category_id_MLA9990,category_id_MLA9991,category_id_MLA9992,category_id_MLA9996,category_id_MLA9999,category_id_MLA99991,category_id_MLA99994,target_condition
0,MLA578052519,new,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,MLA581565358,used,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MLA578780872,used,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,MLA581877385,new,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,MLA576112692,used,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,MLA581735252,used,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
99996,MLA579045843,new,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
99997,MLA578170688,used,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99998,MLA575062152,used,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Save dataframe to train

In [61]:
df_to_model_encoded.to_csv('df_to_model_encoded_cat.csv', index=False)